<a href="https://colab.research.google.com/github/wayne0git/tensorflow_basic/blob/main/example/tensorflow_image_classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Image Classification Example
- https://learnopencv.com/tensorflow-lite-model-optimization-for-on-device-machine-learning/

## Import Library

In [1]:
import os
import numpy as np          # 1.21.6

In [2]:
import tensorflow as tf        # 2.8.0
import tensorflow_datasets as tfds

In [3]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization

In [5]:
%load_ext tensorboard

## Load Dataset
- Cat vs Dog dataset from tfds

In [ ]:
# Load data with a split ratio of 0.7 (train) : 0.2 (validation) : 0.1 (test)
(train_ds, val_ds, test_ds), info = tfds.load('cats_vs_dogs',\
                          split=['train[:70%]', 'train[70%:90%]', 'train[90%:]'],\
                          shuffle_files=True, as_supervised=True, with_info=True)

In [14]:
# Check dataset number
print("Training Images: %d" % tf.data.experimental.cardinality(train_ds).numpy())
print("Validation Images: %d" % tf.data.experimental.cardinality(val_ds).numpy())
print("Test Images: %d" % tf.data.experimental.cardinality(test_ds).numpy())

Training Images: 16283
Validation Images: 4653
Test Images: 2326


In [9]:
# Check label info
print("Number of  Classes: " + str(info.features['label'].num_classes))
print("Classes : " + str(info.features['label'].names))

Number of  Classes: 2
Classes : ['cat', 'dog']


In [ ]:
# Check image data
vis = tfds.visualization.show_examples(train_ds, info)

## Data Preprocessing

In [16]:
# Parameter
N_BATCH = 16
IMG_SIZE = [224, 224]

In [17]:
# Resize image data
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))
val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y))

In [18]:
# Buffering the dataset
train_ds = train_ds.cache().batch(N_BATCH).prefetch(buffer_size=10)
val_ds = val_ds.cache().batch(N_BATCH).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(N_BATCH).prefetch(buffer_size=10)

## Build Model

In [19]:
# Parameter
INPUT_SIZE = (224, 224, 3)

In [23]:
# EfficientNetB0
eff_net = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SIZE, pooling='max')

In [24]:
# Adding layer for transfer learning
x = Dense(512, activation='relu')(eff_net.output)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# Build image classification model
model = Model(inputs=eff_net.input, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=["accuracy"])
model.summary()

## Train Model

In [31]:
# Parameter
MODEL_FPATH = '/content/train/model.h5'
N_EPOCH = 15

In [29]:
# Callback to save best model
model_save = tf.keras.callbacks.ModelCheckpoint(MODEL_FPATH,
                         monitor="val_accuracy",
                         verbose=0,
                         save_best_only=True,
                         save_weights_only=False,
                         mode="max",
                         save_freq="epoch")

In [30]:
# Callback to Reduce LR Callback reduces the learning rate if validation loss remains the same for 3 consecutive epochs.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                         factor=0.1, 
                         patience=3, 
                         verbose=1, 
                         min_delta=5*1e-3,
                         min_lr=5*1e-9,)

In [ ]:
model.fit(train_ds, epochs=N_EPOCH, 
     steps_per_epoch=(len(train_ds)//N_BATCH),
     validation_data=val_ds, validation_steps=(len(val_ds)//N_BATCH),
     shuffle=False, callbacks=[reduce_lr, model_save])

## Evaluate Model

In [ ]:
_, baseline_model_accuracy = model.evaluate(test_ds, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)